# Computação Gráfica 2025/2026

## Aula TP3 - Projecções em 3D com matplotlib e numpy
### Um tutorial gráfico por André Falcão


Vamos aqui usar um módulo do matplotlib (`mpl_toolkits.mplot3d`) para fazer plots em 3D. 
Esta biblioteca simplifica os processos de criação de perspectivas entre outras coisas, que só serão abordadas para a semana

Esta biblioteca de alto nível obriga à utilização de mecanismos específicos para desenho, que não entraremos em detalhe. Assumindo a função de desenho  `draw_scene()` que recebe as dimensões do cubo de visão (`L`), a dimensão por omissão da figura (`figsize`) e o ponto de aobservação, definido como uma elevação (`elev`) face ao (0,0,0) e um  azimute relativamente ao eixo dos x (`azim`)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

def _map_world_to_mpl(P): 
    X, Y, Z = P[...,0], P[...,1], P[...,2] 
    return np.stack([X, -Z, Y], axis=-1)

def draw_scene(faces_world, L=5.0, figsize=(7,6), elev=5, azim=-50):

    faces_mpl = [_map_world_to_mpl(np.asarray(F, float)) for F in faces_world]

    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111, projection='3d')
    ax.set_proj_type("persp")        # perspectiva (opcional)

    # Objecto (uma única Poly3DCollection)
    obj = Poly3DCollection(faces_mpl, facecolors=(0.75,0.78,1.0,0.45),
                           edgecolors='k', linewidths=1.0)
    ax.add_collection3d(obj)

    # Eixos do mundo (no ecra): X→, Y↑, Z→câmara
    ax.quiver(0,0,0,  L,0,0,  arrow_length_ratio=0.05, color='r')  # X →
    ax.text(L,0,0,'X', color='r')
    ax.quiver(0,0,0,  0,0,L,  arrow_length_ratio=0.05, color='g')  # Y ↑
    ax.text(0,0,L,'Y', color='g')
    ax.quiver(0,0,0,  0,-L,0, arrow_length_ratio=0.05, color='b')  # Z → câmara
    ax.text(0,-L,0,'Z', color='b')

    # Cubo de vista [-L, L] em todos os eixos e aspecto 1:1:1
    ax.set_xlim(-L, L); ax.set_ylim(-L, L); ax.set_zlim(-L, L)
    ax.set_box_aspect([1,1,1])

    # Fixar a câmara para respeitar a convenção X→, Y↑, Z→câmara
    ax.view_init(elev=elev, azim=azim)

    # Rótulos coerentes com a convenção (nota: mpl-Y é profundidade)
    ax.set_xlabel('X')   # direita
    ax.set_ylabel('Z')   # profundidade (Z→câmara é -Ympl)
    ax.set_zlabel('Y')   # cima

    plt.tight_layout()
    plt.show()



### Os objectos da cena 

Vamos usar um cubo simples, criando u ma função para desenhar cubos, dado um ponto central e um lado. A função define as 6 faces do cubo como polígonos separados

Criaremos para já um cubo com centro em `[2, -1, 0]` e lado = 1.0

In [ ]:

def faz_cubo(centro, lado):
    r = lado/2
    offs = np.array([[+r,+r,+r], [+r,+r,-r], [+r,-r,+r], [+r,-r,-r],
                     [-r,+r,+r], [-r,+r,-r], [-r,-r,+r], [-r,-r,-r]])
    Vm = centro + offs               # (mundo)
    # faces (mesmo índice, mas em coords mapeadas)
    return [ np.array([Vm[0], Vm[2], Vm[3], Vm[1]]),
              np.array([Vm[4], Vm[5], Vm[7], Vm[6]]),
              np.array([Vm[0], Vm[1], Vm[5], Vm[4]]),
              np.array([Vm[2], Vm[6], Vm[7], Vm[3]]),
              np.array([Vm[0], Vm[4], Vm[6], Vm[2]]),
              np.array([Vm[1], Vm[3], Vm[7], Vm[5]])]
    
centro = np.array([2.0, -1.0, 0.0])  # mundo (X,Y,Z)
lado = 1.0

cubo= faz_cubo(centro,lado)
draw_scene(cubo, L=5.0)


#### Exercícios 1
1. Preste atenção à forma como o cubo foi construído. Veja se compreende a estrutura do objecto
    1. Há vertices repetidos? Porquê?
    2. Será possível estender esta estrutura para colocar mais objectos na cena? discuta como?
2. Experimente mudar a dimensão dos lados e a posição do centro, para ter uma ideia do efeito na posição do cubo na imagem
3. Crie uma função `faz_piramide(lado, altura)` que constroi uma pirâmide de 4 lados em 3D e orientada com o vértice a subir no eixo dos y. Represente-a graficamente 
4. Na função `draw_scene` examine os parâmetros elev e azim (por omissão `elev=5, azim=-50`) que descrevem a posição da câmara. experimente mudá-los e veja o que acontece à cena

In [ ]:
## exercício 2

#FAZER AQUI


In [ ]:
## exercício 3

#FAZER AQUI


In [ ]:
## exercício 3

#FAZER AQUI


## Transformações Geométricas em 3D

como os objectos não estão nativamente em coordenadas homogéneas, vamos precisar de alguns utilitários para a conversão entre formatos, para aplicar as transformações geométricas, mas depois para usar as funções de desenho do matplotlib



In [ ]:
def to_homog_faces(faces_xyz):
    """(list de Ni×3) -> (list de Ni×4) adicionando coluna w=1."""
    return [np.c_[f, np.ones((len(f), 1))] for f in faces_xyz]

def from_homog_faces(faces_h):
    """(list de Ni×4) -> (list de Ni×3), faz divisão por w."""
    
    out = []
    for fh in faces_h:
        w = fh[:, [3]]
        out.append(fh[:, :3] / w)
    return out


Aqui podemos finalmente definir as várias matrizes de transformações que iremos necessitar:
* Translações (`T`)
* Escalamentos (`S`)
* Rotações (`Rx`, `Ry`, `Rz`)

**Um aspecto importante:** Uma vez que os vértices da nossa cena vão aparecer em linhas, as matrizes terão que ser transpostas e aplicadas na ordem inversa da explicada nas aulas teóricas, que assume que os pontos aparecem em coluna. Todo o raciocínio é absolutamente idêntico e a ordem, apesar de aparentemente inversa, não se altera

In [ ]:

def T(tx, ty, tz):
    """Translação."""
    M = np.eye(4, dtype=float)
    M[:3, 3] = [tx, ty, tz]
    return M

def S(sx, sy, sz):
    """Escalamento em torno da origem."""
    M = np.eye(4, dtype=float)
    M[0,0], M[1,1], M[2,2] = sx, sy, sz
    return M

def Rx(theta, degrees=True):
    """Rotação em torno de X (RH)."""
    if degrees: theta = np.deg2rad(theta)
    c, s = np.cos(theta), np.sin(theta)
    M = np.array([[1, 0, 0, 0],
                  [0, c,-s, 0],
                  [0, s, c, 0],
                  [0, 0, 0, 1]], dtype=float)
    return M

def Ry(theta, degrees=True):
    """Rotação em torno de Y (RH)."""
    if degrees: theta = np.deg2rad(theta)
    c, s = np.cos(theta), np.sin(theta)
    M = np.array([[ c, 0, s, 0],
                  [ 0, 1, 0, 0],
                  [-s, 0, c, 0],
                  [ 0, 0, 0, 1]], dtype=float)
    return M

def Rz(theta, degrees=True):
    """Rotação em torno de Z (RH)."""
    if degrees: theta = np.deg2rad(theta)
    c, s = np.cos(theta), np.sin(theta)
    M = np.array([[c,-s, 0, 0],
                  [s, c, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]], dtype=float)
    return M


#composição de matrizes
def compose(Ms):
    Mtot = np.eye(4, dtype=float)
    for M in Ms:
        Mtot = Mtot @ M
    return Mtot

def apply_transform_h(cena_xyz, M):
    
    #primeiro converte os objectos da cena para coordenadas homogéneas
    Fh = to_homog_faces(cena_xyz) 
    #repare-se <- Os objectos da cena aparecem primeiro e depois a matriz é transposta antes da aplicação
    return [fh @ M.T for fh in Fh]



### Alguns exemplos

Comecemos por criar um cubo de lado =1.5 e com centro em (3,0,0) (eixo dos xx
Dep

In [ ]:
centro = np.array([3.0, 0.0, 0.0])  # mundo (X,Y,Z)
lado = 1.5

cubo= faz_cubo(centro,lado)
draw_scene(cubo, figsize=(4,4))

### Translação

Vamos deslocar este objecto para o eixo dos zz, para isso diminuimos 3 no x e somamos 3 no z

In [ ]:

Mt = T(-3, 0, 3)
cubo_t  = apply_transform_h(cubo, Mt)
draw_scene(cubo_t, L=5.0, figsize=(4,4))


### Escalamentos

Vamos alongar o nosso cubo no eixo do x e colocá-lo mais pequeno no y e z

In [ ]:
Ms= S(1.5,.2,.5)
#Mrx= Rx(30)
cubo_s = apply_transform_h(cubo, Ms)
draw_scene(cubo_s, L=5.0, figsize=(4,4))

#### Composições ad-hoc

Note-se que podemos fazer composições de transformações geométricas (e já vimos duas, translações e escalamentos) aplicando sucessivamente transformações aos produtos transformados. Mas atenção! veja-se que os resultados das transformações estão em coordenadas homogéneas, e as nossas transformações estão a ser aplicadas em coordenadas originais da cena, pelo que teremos que converter as nossas coordenadas para as originais.  **Este procedimento é um "hack" devido às limitações do sistema que temos, essencialmente académico e demonstrativo!**

Vamos usar este "hack" para deslocar a nossa "caixa" definida acima para outra posição


In [ ]:
#primeiro convertemos as coordenadas para xyz
caixa = from_homog_faces(cubo_s)

#vamos elevar a caixa no eixo dos y - primeiro criar a matriz
Mt= T(0,3,0)
#agora aplicamos a translação 
caixa_st = apply_transform_h(caixa, Mt)

#e desenhar o resultado
draw_scene(caixa_st, L=5.0, figsize=(4,4))

### Rotações

Podemos aplicar rotações nos 3 eixos (x, y e z) com as matrizes que conhecemos

Vamos aplicar as várias transformações à caixa previamente gerada, pois é mais interessante para aplicar e visualisar as transformações

In [ ]:


Mrx= Rx(60)
x = apply_transform_h(caixa, Mrx)
draw_scene(x, figsize=(4,4))

In [ ]:
Mry= Ry(-60)
x = apply_transform_h(caixa, Mry)
draw_scene(x, figsize=(4,4))

In [ ]:
Mrz= Rz(30)
x = apply_transform_h(caixa, Mrz)
draw_scene(x, figsize=(4,4))

#### Exercícios 2

1. Experimente vários tipos de rotações da caixa. Verifique os sentidos directos e inversos nos ângulos
2. Como colocaria a caixa no eixo do y? e no eixo do z?
3. Usando o "hack" descrito acima, consegue centrar a caixa no centro do gráfico (0,0,0) e reorientar a caixa para que fique a apontar o seu comprimento no eixo do z?

In [ ]:
# Exercício 1

#FAZER AQUI

In [ ]:
# Exercício 2

#FAZER AQUI

In [ ]:
# Exercício 3

#FAZER AQUI

### Rotações sobre eixos arbitrários

Rotações sobre eixos arbitrários são operações mais complexas, obrigando a uma composição de 7 transformações. 

Sendo um eixo definido com uma origem (P) e um vector (V), para fazer uma rotação de $\theta$ sobre esse eixo, as operações são as seguintes:

1. definir a matriz de translação na qual P se anula
2. definir a matriz de rotação sobre um eixo, que anula uma das coordenadas do vector, projectando o vector sobre o plano (geralmente xz ou yz)
3. definir a matriz de rotação que coloca o o vector num dos eixos (geralmente o eixo dos zz) anulando a segunda coordenada do vector
4. definir a matriz de rotação que faz a rotação de $\theta$ sobre o eixo definido
5. definir a matriz de rotação que anula a rotação de 3
6. definir a matriz de rotação que anula a rotação de 2
7. definir a matriz de translação inversa de 1

Estas 7 matrizes são multiplicadas pela ordem definida

Vamos primeiro definir os eixos de rotação com um ponto e vector


In [ ]:

# dados do eixo/ponto/ângulo
u = np.array([1.0, 1.0, 0.0])        # direção do eixo
c = np.array([0.0, 1.0, 1.0])        # ponto por onde passa

#valores auxiliares
ux, uy, uz = u
L = np.hypot(ux, uy)     #(porque iremos projectar em zz)            



Como vimos nas aulas teóricas podemos precalcular os ângulos que iremos necessitar

$$ \alpha =arctan2(-u_y, u_x) = arctan2(-1, 1) = -\frac{\pi}{4} $$

$$ \beta =arctan2(-L, u_z) = arctan2(-\sqrt{2}, 0) =  -\frac{\pi}{2} $$


In [ ]:

alpha = np.degrees(np.arctan2(-uy, ux))      
beta  = np.degrees(np.arctan2(-L, uz))  
alpha, beta

In [ ]:
theta = -45.0                      # rotação final (graus)

# matrizes (APENAS T/Rz/Ry) — ordem exatamente como descrita

# 1 matriz de translação na qual P se anula
M1 = T(-c[0], -c[1], -c[2])

#matriz rotação sobre z que anula y projectando o vector sobre o plano xz 
M2 = Rz(alpha)

#matriz rotação sobre y que anula x projectando o vector sobre o eixo z 
M3 = Ry(beta)

# rotação efetiva em torno do eixo alinhado a +Z
M4 = Rz(theta)     

#desfazer beta
M5 = Ry(-beta)

#desfazer alpha
M6 = Rz(-alpha)

#Desfazer translação
M7 = T(+c[0], +c[1], +c[2])

#compor a matriz final (sem hack)
M = compose([M7, M6, M5, M4, M3, M2, M1]) 
np.set_printoptions(precision=3, suppress=True)
M.T

Agora podemos aplicar a transformação e verificar o que acontece

In [ ]:

caixa_t   = apply_transform_h(caixa, M)   
draw_scene(caixa_t, figsize=(4,4))  


#### Exercícios 3

1. No exemplo acima, verifique visualmente e quantitativamente que coordenadas são anuladas depois de cada rotação 
2. Experimente fazer rotações arbitrárias mas em que os vectores e pontos de origem correspondem aos eixos principais
    1. Quais serão as coordenadas de P e V para cada um dos eixos?
    2. Verifique que o resultado é o esperado
3. Faça uma rotação arbitrária de 15 graus sobre o eixo defindido como P(-2,3,1) e V (0, -1, -3)
    1. verifique o que acontece à caixa
    2. verifique que ao fim de aplicar as matrizes até M3, o vector V está de facto projectado sobre z
4. Crie uma função `R_axis` que aceite um ponto, um vector e um ângulo theta e devolva a matriz final pronta para transformar a cena
    1. Teste a a sua função comparando-a com os resultados anteriores
4. [OPCIONAL] Veja a transformação de Rodrigues e implemente-a para fazer o mesmo que a sua função definida acima faz
    1. Compare o desempenho de ambas as implementações
5. [OPCIONAL] modifique a função `draw_scene` para que possa receber mais do que um objecto e cores e use-a para representar os objectos desta aula, antes e depois das transformações   
   

In [ ]:
# Exercício 1

#FAZER AQUI

In [ ]:
# Exercício 2

#FAZER AQUI <- testar os 3 eixos

In [ ]:
# Exercício 3

#FAZER AQUI

In [ ]:
# Exercício 4

#FAZER AQUI

In [ ]:
# Exercício 5 [opcional]

#FAZER AQUI

In [ ]:
# Exercício 5 [opcional]

#FAZER AQUI